In [1]:
exec(open("init_notebook.py", "r").read())

current working dir/home/abjawad/Documents/GitHub/local-attention-model


In [2]:
from models.swin_transformer import PatchEmbed
from models.encoders.student_segformer import OverlapPatchEmbed

[Warning] Fused window process have not been installed. Please refer to get_started.md for installation.


In [3]:
swin_patch_embed = PatchEmbed(img_size=224, 
                              patch_size=4, 
                              in_chans=3, 
                              embed_dim=96, 
                              norm_layer=None)

overlap_patch_embed = OverlapPatchEmbed(patch_size=7, 
                                        stride=4, 
                                        in_chans=3, 
                                        embed_dim=768)


##### self.patch_embed1 = OverlapPatchEmbed(patch_size=7, stride=4, in_chans=in_chans, embed_dim=64)

In [4]:
print(swin_patch_embed.flops())
print(overlap_patch_embed.flops(1024, 1024))

14450688
119990648832
